## Sampling & Splitting Data

## Part A: 분석을 위한 환경 설정

### 모듈/패키지 로드

In [1]:
evaluation = True
evaluation_verbose = False

OUTPUT_BUCKET_FOLDER = "gs://cap-18/output/"
DATA_BUCKET_FOLDER = "gs://cap-18/data/"

In [2]:
from IPython.display import display

In [3]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.linalg import Vectors, SparseVector, VectorUDT

In [4]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" # 한 셀(cell)에서의 코드 실행 결과가 다 보이도록 설정

In [5]:
import numpy as np
import scipy.sparse

In [6]:
import warnings 
warnings.filterwarnings('ignore') # warning 메시지 반환되지 않도록 설정

In [7]:
import math
import datetime
import time
import itertools

In [8]:
import pickle

In [9]:
import random
random.seed(42)

In [11]:
import pandas as pd
%matplotlib inline

## Part B: 변수별 결측치 확인

In [8]:
# 테이블 로드
train_valid_merged_df = spark.read.parquet("gs://cap-18/output/train_subset_final")

In [12]:
## cf) 판다스 데이터 프레임 구조 깨지지 않게 보기:
pd.set_option('display.max_columns', 100)

In [10]:
documents_categories_schema = StructType(
                    [StructField("document_id_cat", IntegerType(), True),
                    StructField("category_id", IntegerType(), True),                    
                    StructField("confidence_level_cat", FloatType(), True)]
                    )

documents_categories_df = spark.read.schema(documents_categories_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER+"documents_categories.csv") \
                .alias('documents_categories').cache()
    
documents_categories_grouped_df = documents_categories_df.groupBy('document_id_cat') \
                                            .agg(F.collect_list('category_id').alias('category_id_list'),
                                                 F.collect_list('confidence_level_cat').alias('confidence_level_cat_list')) \
                                            .withColumn('dummyDocumentsCategory', F.lit(1)) \
                                            .alias('documents_categories_grouped')

In [13]:
documents_topics_schema = StructType(
                    [StructField("document_id_top", IntegerType(), True),
                    StructField("topic_id", IntegerType(), True),                    
                    StructField("confidence_level_top", FloatType(), True)]
                    )

documents_topics_df = spark.read.schema(documents_topics_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER+"documents_topics.csv")  \
                .alias('documents_topics').cache()
    
documents_topics_grouped_df = documents_topics_df.groupBy('document_id_top') \
                                            .agg(F.collect_list('topic_id').alias('topic_id_list'),
                                                 F.collect_list('confidence_level_top').alias('confidence_level_top_list')) \
                                            .withColumn('dummyDocumentsTopics', F.lit(1)) \
                                            .alias('documents_topics_grouped')

다음의 코드로 변수별 결측치를 한번에 확인할 수 있다.

In [14]:
## cf) 판다스 100행까지 보기:
pd.set_option('display.max_rows', 100)

In [19]:
# train_valid_merged_df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in train_valid_merged_df.columns]).toPandas().transpose()

,0
display_id,0
ad_id,0
label,0
doc_id,0
is_leak,0
event_weekend,0
user_views,55311626
ad_views,33833186
doc_views,52700407
doc_event_hour,0


`user_views`, `ad_views`, `doc_views`는 참조할 사전 데이터가 없는 경우에 발생하기 때문에 1로 채워넣었다.

In [15]:
train_imputing_df = train_valid_merged_df

In [16]:
train_imputing_df = train_imputing_df.na.fill({'user_views': 1, 'ad_views': 1, 'doc_views': 1})

`traffic_source`와 `platform_event`는 최빈값인 1로 채우자.

In [17]:
train_imputing_df = train_imputing_df.na.fill({'traffic_source': 1, 'platform_event': 1})

In [18]:
train_imputing_df = train_imputing_df.withColumnRenamed('doc_event_source_id', 'view_source_id') \
                                    .withColumnRenamed('doc_event_publisher_id', 'view_publisher_id') \
                                    .drop('platform_event')

In [19]:
train_imputing_df.columns

['display_id',
 'ad_id',
 'label',
 'doc_id',
 'is_leak',
 'event_weekend',
 'user_views',
 'ad_views',
 'doc_views',
 'doc_event_hour',
 'pop_ad_id',
 'pop_ad_id_conf',
 'pop_ad_id_conf_multipl',
 'pop_document_id',
 'pop_document_id_conf',
 'pop_document_id_conf_multipl',
 'pop_publisher_id',
 'pop_publisher_id_conf',
 'pop_publisher_id_conf_multipl',
 'pop_advertiser_id',
 'pop_advertiser_id_conf',
 'pop_advertiser_id_conf_multipl',
 'pop_campaign_id',
 'pop_campaign_id_conf',
 'pop_campaign_id_conf_multipl',
 'pop_doc_event_doc_ad',
 'pop_doc_event_doc_ad_conf',
 'pop_doc_event_doc_ad_conf_multipl',
 'pop_source_id',
 'pop_source_id_conf',
 'pop_source_id_conf_multipl',
 'pop_source_id_country',
 'pop_source_id_country_conf',
 'pop_source_id_country_conf_multipl',
 'pop_topic_id',
 'pop_topic_id_conf',
 'pop_topic_id_conf_multipl',
 'pop_category_id',
 'pop_category_id_conf',
 'pop_category_id_conf_multipl',
 'pop_category_id_country',
 'pop_category_id_country_conf',
 'pop_categor

pop-으로 시작하는 CTR과 그 외 범주형 변수의 결측치는 알려지지 않음(U: Unknown)으로 대체하자.

In [20]:
train_imputing_df = train_imputing_df.na.fill({'ad_publisher_id': 'U', 'ad_source_id': 'U', 'view_topic_id': 'U',
                                              'view_category_id': 'U', 'ad_topic_id': 'U', 'ad_category_id': 'U', 'view_category_conf' : 'U', 
                                               'ad_topic_conf' : 'U', 'ad_category_conf': 'U',
                                              'view_publisher_id' : 'U', 'view_source_id' : 'U',
                                              'pop_category_id_country_conf_multipl' : 'U',
                                              'pop_category_id_country_conf' : 'U',
                                               'pop_category_id_country' : 'U',
                                               'pop_category_id_conf_multipl' : 'U',
                                               'pop_category_id_conf' : 'U',
                                               'pop_category_id' : 'U',
                                               'pop_topic_id_conf_multipl' : 'U',
                                               'pop_topic_id_conf' : 'U',
                                               'pop_topic_id' : 'U',
                                               'pop_source_id_country_conf_multipl' : 'U',
                                               'pop_source_id_country_conf' : 'U',
                                               'pop_source_id_country' : 'U',
                                               'pop_source_id_conf_multipl' : 'U',
                                               'pop_source_id_conf' : 'U',
                                               'pop_source_id' : 'U',
                                               'pop_doc_event_doc_ad_conf_multipl' : 'U',
                                               'pop_doc_event_doc_ad_conf' : 'U',
                                               'pop_doc_event_doc_ad' : 'U',
                                               'pop_campaign_id_conf_multipl' : 'U',
                                               'pop_campaign_id_conf' : 'U',
                                               'pop_campaign_id' : 'U',
                                               'pop_advertiser_id_conf_multipl' : 'U',
                                               'pop_advertiser_id_conf' : 'U',
                                               'pop_advertiser_id' : 'U',
                                               'pop_publisher_id_conf_multipl' : 'U',
                                               'pop_publisher_id_conf' : 'U',
                                               'pop_publisher_id' : 'U',
                                               'pop_document_id_conf_multipl' : 'U',
                                               'pop_document_id_conf' : 'U',
                                               'pop_document_id' : 'U',
                                               'pop_ad_id_conf_multipl' : 'U',
                                               'pop_ad_id_conf' : 'U',
                                               'pop_ad_id' : 'U'
                                              })

# ad_publisher_id 유난히 NA가 많음 => 확인 필요 => okay. 원래 데이터에서도 그렇다.
# document_id_top, document_id_cat 삭제 => okay. Feature Enginnering_05 코드에 반영하였음. 재실행 시 해당 변수가 사라져 있을 것!

## train set과 valid set 나누기
train set과 valid set 선언, size 확인<br>
train set, valid set을 timestamp_event 순서대로 정렬<br>
monotonically_increasing_id 사용하여 index 컬럼 생성<br>
train set은 순서대로 80000개 끊어서 샘플 데이터(train_merged_df_sample) 생성<br>
valid set은 순서대로 36653개 끊어서 샘플 데이터(valid_merged_df_sample) 생성

In [21]:
train_merged_df = train_imputing_df.filter(train_imputing_df.is_train == 1)
valid_merged_df = train_imputing_df.filter(train_imputing_df.is_train == 0)

In [23]:
print(train_merged_df.count()) # 68.58%
print(valid_merged_df.count()) # 31.42%
print(train_imputing_df.count())

59761474
27380257
87141731


train

In [22]:
train_merged_df_ordered = train_merged_df.orderBy("timestamp_event")

In [23]:
from pyspark.sql.functions import monotonically_increasing_id
train_merged_df_ordered_index = train_merged_df_ordered.withColumn("index", monotonically_increasing_id())

In [24]:
train_merged_df_sample = train_merged_df_ordered_index.filter(train_merged_df_ordered_index.index < 80000)

In [27]:
train_merged_df_sample.count() # 80000 / 59761474 = 0.00133865507

80000

valid

In [25]:
valid_merged_df_ordered = valid_merged_df.orderBy("timestamp_event")
valid_merged_df_ordered_index = valid_merged_df_ordered.withColumn("index", monotonically_increasing_id())

In [26]:
valid_merged_df_sample = valid_merged_df_ordered_index.filter(valid_merged_df_ordered_index.index < 36653)

In [30]:
valid_merged_df_sample.count() # 27380257 * 0.00133865507 = 36652.719851

36653

## train set, valid set에서 필요 없는 컬럼 버리기

In [27]:
train_merged_df_sample.columns

['display_id',
 'ad_id',
 'label',
 'doc_id',
 'is_leak',
 'event_weekend',
 'user_views',
 'ad_views',
 'doc_views',
 'doc_event_hour',
 'pop_ad_id',
 'pop_ad_id_conf',
 'pop_ad_id_conf_multipl',
 'pop_document_id',
 'pop_document_id_conf',
 'pop_document_id_conf_multipl',
 'pop_publisher_id',
 'pop_publisher_id_conf',
 'pop_publisher_id_conf_multipl',
 'pop_advertiser_id',
 'pop_advertiser_id_conf',
 'pop_advertiser_id_conf_multipl',
 'pop_campaign_id',
 'pop_campaign_id_conf',
 'pop_campaign_id_conf_multipl',
 'pop_doc_event_doc_ad',
 'pop_doc_event_doc_ad_conf',
 'pop_doc_event_doc_ad_conf_multipl',
 'pop_source_id',
 'pop_source_id_conf',
 'pop_source_id_conf_multipl',
 'pop_source_id_country',
 'pop_source_id_country_conf',
 'pop_source_id_country_conf_multipl',
 'pop_topic_id',
 'pop_topic_id_conf',
 'pop_topic_id_conf_multipl',
 'pop_category_id',
 'pop_category_id_conf',
 'pop_category_id_conf_multipl',
 'pop_category_id_country',
 'pop_category_id_country_conf',
 'pop_categor

In [28]:
drop_cols = ['display_id', 'ad_id', 'doc_id', 'doc_event_hour', 'pop_ad_id_conf', 'pop_ad_id_conf_multipl', 'pop_document_id_conf', 'pop_document_id_conf_multipl', 'pop_publisher_id_conf', 'pop_publisher_id_conf_multipl', 'pop_advertiser_id_conf', 'pop_advertiser_id_conf_multipl', 'pop_campaign_id_conf', 'pop_campaign_id_conf_multipl', 'pop_doc_event_doc_ad_conf', 'pop_doc_event_doc_ad_conf_multipl', 'pop_source_id_conf', 'pop_source_id_conf_multipl', 'pop_source_id_country', 'pop_source_id_country_conf', 'pop_source_id_country_conf_multipl', 'pop_topic_id_conf', 'pop_topic_id_conf_multipl', 'pop_category_id_conf', 'pop_category_id_conf_multipl', 'pop_category_id_country', 'pop_category_id_country_conf', 'pop_category_id_country_conf_multipl', 'campaign_id', 'is_train', 'uuid', 'day_event', 'view_doc_id', 'ad_doc_id', 'ad_category_conf', 'ad_topic_conf', 'view_category_conf', 'view_topic_conf', 'document_id_cat', 'document_id_top', 'timestamp_event', 'index']
train_merged_df_sample = train_merged_df_sample.drop('display_id', 'ad_id', 'doc_id', 'doc_event_hour', 'pop_ad_id_conf', 'pop_ad_id_conf_multipl', 'pop_document_id_conf', 'pop_document_id_conf_multipl', 'pop_publisher_id_conf', 'pop_publisher_id_conf_multipl', 'pop_advertiser_id_conf', 'pop_advertiser_id_conf_multipl', 'pop_campaign_id_conf', 'pop_campaign_id_conf_multipl', 'pop_doc_event_doc_ad_conf', 'pop_doc_event_doc_ad_conf_multipl', 'pop_source_id_conf', 'pop_source_id_conf_multipl', 'pop_source_id_country', 'pop_source_id_country_conf', 'pop_source_id_country_conf_multipl', 'pop_topic_id_conf', 'pop_topic_id_conf_multipl', 'pop_category_id_conf', 'pop_category_id_conf_multipl', 'pop_category_id_country', 'pop_category_id_country_conf', 'pop_category_id_country_conf_multipl', 'campaign_id', 'is_train', 'uuid', 'day_event', 'view_doc_id', 'ad_doc_id', 'ad_category_conf', 'ad_topic_conf', 'view_category_conf', 'view_topic_conf', 'document_id_cat', 'document_id_top', 'timestamp_event', 'index')
valid_merged_df_sample = valid_merged_df_sample.drop('display_id', 'ad_id', 'doc_id', 'doc_event_hour', 'pop_ad_id_conf', 'pop_ad_id_conf_multipl', 'pop_document_id_conf', 'pop_document_id_conf_multipl', 'pop_publisher_id_conf', 'pop_publisher_id_conf_multipl', 'pop_advertiser_id_conf', 'pop_advertiser_id_conf_multipl', 'pop_campaign_id_conf', 'pop_campaign_id_conf_multipl', 'pop_doc_event_doc_ad_conf', 'pop_doc_event_doc_ad_conf_multipl', 'pop_source_id_conf', 'pop_source_id_conf_multipl', 'pop_source_id_country', 'pop_source_id_country_conf', 'pop_source_id_country_conf_multipl', 'pop_topic_id_conf', 'pop_topic_id_conf_multipl', 'pop_category_id_conf', 'pop_category_id_conf_multipl', 'pop_category_id_country', 'pop_category_id_country_conf', 'pop_category_id_country_conf_multipl', 'campaign_id', 'is_train', 'uuid', 'day_event', 'view_doc_id', 'ad_doc_id', 'ad_category_conf', 'ad_topic_conf', 'view_category_conf', 'view_topic_conf', 'document_id_cat', 'document_id_top', 'timestamp_event', 'index')

In [29]:
print(len(train_merged_df_ordered_index.columns)) # 71
print(len(train_merged_df_sample.columns)) # 48

70
28


In [30]:
train_merged_df_sample.columns

['label',
 'is_leak',
 'event_weekend',
 'user_views',
 'ad_views',
 'doc_views',
 'pop_ad_id',
 'pop_document_id',
 'pop_publisher_id',
 'pop_advertiser_id',
 'pop_campaign_id',
 'pop_doc_event_doc_ad',
 'pop_source_id',
 'pop_topic_id',
 'pop_category_id',
 'view_publisher_id',
 'view_source_id',
 'event_hour',
 'event_platform',
 'traffic_source',
 'event_country',
 'advertiser_id',
 'ad_category_id',
 'ad_topic_id',
 'view_category_id',
 'view_topic_id',
 'ad_source_id',
 'ad_publisher_id']

In [31]:
train_merged_df_sample.take(1)

[Row(label=0, is_leak=-1, event_weekend=0, user_views=1, ad_views=1, doc_views=1, pop_ad_id=0.032326001673936844, pop_document_id=0.03453899919986725, pop_publisher_id=None, pop_advertiser_id=0.04069000110030174, pop_campaign_id=0.034035999327898026, pop_doc_event_doc_ad=None, pop_source_id=0.03703000023961067, pop_topic_id=0.1357100009918213, pop_category_id=0.11432000249624252, view_publisher_id=24, view_source_id=6482, event_hour=6, event_platform=3, traffic_source=1, event_country='US', advertiser_id=1919, ad_category_id='1503', ad_topic_id='138', view_category_id='1203', view_topic_id='285', ad_source_id=8064, ad_publisher_id=None)]

In [32]:
len(train_merged_df_sample.columns)

28

In [33]:
%time train_merged_df_sample.write.csv("gs://capstone-02/output/train-80000")

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 53.3 s


In [36]:
%time valid_merged_df_sample.write.csv("gs://capstone-02/output/valid-80000")

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 20 s
